In [1]:
import pickle
import torch
import numpy as np
import MF
import math
from sklearn import cluster
from operator import itemgetter
from math import log
import torch.nn as nn
import torch.nn.functional as F
import time
import argparse
import pandas as pd
import utility
from scipy.sparse import csr_matrix, rand as sprand
from tqdm import tqdm
from MF import MF
import VAE
import os


np.random.seed(0)
torch.manual_seed(0)

In [7]:
### Unpickling user and item matrices

test_like = np.load("./data/user_vali_like.npy", allow_pickle=True)
train_like = np.load("./data/user_train_like.npy", allow_pickle=True)

### mtx_item[I]: the list of users that like item I
### mtx_user[U]: the list of items that user U like

with open("./data/item", "rb") as f: 
    mtx_item = pickle.load(f)

with open("./data/user", "rb") as f:
    mtx_user = pickle.load(f)
    
### Read the trained model
model = torch.load("./data/save.pt")
# Item matrix
Q = model.item_factors.weight.H.tolist()
# User matrix
P = model.user_factors.weight.H.tolist()

P = np.array(P)
Q = np.array(Q)

### Predicted matrix
Rec = np.matmul(P.T, Q)


### The test_like and train_like lists, but for item
### test_like_item[I]: the users who like item I in the test set
### train_like_item[I]: the users who like item I in the train set
test_like_item = []
for t in range(Rec.shape[1]):
    test_like_item.append([])
i = 0
for t in test_like:
    for tt in t:
        test_like_item[tt].append(i)
    i += 1

train_like_item = []
for t in range(Rec.shape[0]):
    train_like_item.append([])
i = 0
for t in train_like:
    for tt in t:
        train_like_item[tt].append(i)
    i += 1
    
Rec_train = []
for t in range(Rec.shape[0]):
    row = [0]*Rec.shape[1]
    for item in train_like[t]:
        row[item] = 1
    Rec_test.append(row)
Rec_train = np.array(Rec_train)

J_similar = Rec_train.dot(Rec_train.T)
for t in range(J_similar.shape[0]):
    J_similar[t][t] = 0
for i in range(J_similar.shape[0]):
    for j in range(i+1, J_similar.shape[1]):
        if J_similar[i][j] != 0:
            J_similar[i][j] /= (sum(Rec_train[i]) + sum(Rec_train[j]))
for i in range(J_similar.shape[0]):
    for j in range(i):
        J_similar[i][j] = J_similar[j][i]

In [15]:
Rec_train_item = Rec_train.T
J_similar_item = Rec_train_item.dot(Rec_train)
for t in range(J_similar_item.shape[0]):
    J_similar_item[t][t] = 0
for i in range(J_similar_item.shape[0]):
    for j in range(i+1, J_similar_item.shape[1]):
        if J_similar_item[i][j] != 0:
            J_similar_item[i][j] /= (sum(Rec_train_item[i]) + sum(Rec_train_item[j]))
for i in range(J_similar_item.shape[0]):
    for j in range(i):
        J_similar[i][j] = J_similar[j][i]

In [21]:
### Functions for metrices
def get_activeness(user):
    return len(train_like[user])

def get_popularity(item):
    return len(train_like_item[item])

def get_user_mainstreamness(user):
    ### mainstreamness == jacarrd similarity
    return(sum(J_similar[user])/(J_similar.shape[0]-1))

def get_item_mainstreamness(item):
    ### mainstreamness == jacarrd similarity
    return(sum(J_similar_item[item])/(J_similar_item.shape[0]-1))



In [22]:
### Calculate the metrices

activeness = []
user_mainstreamness = []
for t in range(Rec.shape[0]):
    activeness.append(get_activeness(t))
    user_mainstreamness.append((get_user_mainstreamness(t)))
                               
popularity = []
item_mainstreamness = []
for t in range(Rec.shape[1]):
    popularity.append(get_popularity(t))
    item_mainstreamness.append((get_item_mainstreamness(t)))

In [24]:
### _sort[0] = values
### _sort[1] = indexs

act_sort = (np.sort(np.array(activeness)), np.argsort(np.array(activeness)))
usrmain_sort = (np.sort(np.array(user_mainstreamness)), np.argsort(np.array(user_mainstreamness)))
pop_sort = (np.sort(np.array(popularity)), np.argsort(np.array(popularity)))
itmmain_sort = (np.sort(np.array(item_mainstreamness)), np.argsort(np.array(item_mainstreamness)))

In [25]:
np.save("./data/activeness.npy", activeness)
np.save("./data/user_mainstreamness.npy", user_mainstreamness)
np.save("./data/popularity.npy", popularity)
np.save("./data/item_mainstreamness.npy", item_mainstreamness)

np.save("./data/act_sort.npy", act_sort)
np.save("./data/usrmain_sort.npy", usrmain_sort)
np.save("./data/pop_sort.npy", pop_sort)
np.save("./data/itmmain_sort.npy", itmmain_sort)

In [14]:
### Load the metrics from disk
act_sort = np.load("./data/act_sort.npy")
pop_sort = np.load("./data/pop_sort.npy")
usrmain_sort = np.load("./data/usrmain_sort.npy")
itmmain_sort = np.load("./data/itmmain_sort.npy")

In [27]:
### Functions for calculating NDCG@K, performance, and average rank

num_u = Rec.shape[0]
like = train_like
for i in range(num_u):
    Rec[i, like[i]] = -100000.0

### No train set in Rec_sort!!!
### Pretty quick even we sort all users and items
Rec_sort = []
for t in Rec:
    num = t.tolist().count(-100000.0)
    tt = np.argsort(t)[::-1]
    tt = tt[0:(len(tt)-num)]
    Rec_sort.append(tt)

def NDCG_at_k(predicted_list, ground_truth, k):
    dcg_value = [(v / log(i + 1 + 1, 2)) for i, v in enumerate(predicted_list[:k])]
    dcg = np.sum(dcg_value)
    if len(ground_truth) < k:
        ground_truth += [0 for i in range(k - len(ground_truth))]
    idcg_value = [(v / log(i + 1 + 1, 2)) for i, v in enumerate(ground_truth[:k])]
    idcg = np.sum(idcg_value)
    return dcg / idcg

def get_ndcg_ave(user):
    k_set = [1,5,10,20]
    u_pred = Rec[user, :]
    top15_item_idx_no_train = np.argpartition(u_pred, -k_set[-1])[-k_set[-1]:]
    top15 = (np.array([top15_item_idx_no_train, u_pred[top15_item_idx_no_train]])).T
    top15 = sorted(top15, key=itemgetter(1), reverse=True)
    new_user_prediction = top15
    test = test_like[user]

    dcg_list = []

    # compute the number of true positive items at top k
    rank_sum = 0
    count = 0
    for i in range(k_set[3]):
        if new_user_prediction[i][0] in test:
            rank_sum += (i+1)
            count += 1
            dcg_list.append(1)
        else:
            dcg_list.append(0)

    # calculate NDCG@k
    idcg_list = [1 for i in range(len(test))]
    ndcg_tmp_15 = NDCG_at_k(dcg_list, idcg_list, k_set[3])
    ### Calculate average rank of test items
    ### Notice: only test items in the top 20 count
    ave_rank = rank_sum/count
    
    return ndcg_tmp_15, ave_rank



def get_performance_ave(item):
### !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
### get performance@20 after 20 is 0
    k = 20
    test = test_like_item[item]
    ### Number of the users who likes this item
    hit_usr = len(test)
    ### Top of the fraction
    a = 0
    
    rank_sum = 0
    count = 0
    for t in range(k):
        if t < hit_usr:
            ranklist = Rec_sort[test[t]]
            rank = ranklist.tolist().index(item)+1
            rank_sum += rank
            count += 1
            a += 1/math.log(rank+2, 2)
        else:
            a += 0
    performance = a/min(hit_usr, k) ### a/min(hit_usr, 20) or a/20?
    ave_rank = rank_sum/count
    
    return performance, ave_rank


In [ ]:
def evaluate_user():
    user_num = Rec.shape[0]
    NDCG = []
    Ave_rank = []
    for t in range(user_num):
        ndcg_, ave_rank_ = get_ndcg_ave(t)
        NDCG.append(ndcg_)
        Ave_rank.append(ave_rank_)
    return NDCG, Ave_rank

def evaluate_item():
    item_num = Rec.shape[1]
    Performance = []
    Ave_rank = []
    for t in range(item_num):
        performance_, ave_rank_ = get_performance_ave(t)
        Performance.append(performance_)
        Ave_rank.append(ave_rank_)
    return Performance, Ave_rank


In [ ]:
NDCG, Ave_rank_user = evaluate_user()
Performance, Ave_rank_item = evaluate_item()

In [18]:
def evaluate_user(mtc, k):
    ### mtc should be index of the users
    
    ### Divide users into k groups evenly
    bt = int(len(mtc[0])/k)
    labels = []
    for t in range(k):
        if t == k-1:
            labels[t*bt::] = [t]*(len(mtc[0])-(k-1)*bt)
        else:
            labels[t*bt:t*(bt+1)] = [t]*bt
        
    NDCG = {}
    Ave_rank = {}
    ave_mtc = {}
    m = {}
    for t in range(k):
        NDCG[t] = []
        Ave_rank[t] = []
        ave_mtc[t] = 0
        n = 0
        m[t] = []
        for ii in range(len(labels)):
            if labels[ii] == t:
                ndcg_, ave_rank_ = get_ndcg_ave(mtc[1][ii])
                NDCG[t].append(ndcg_)
                Ave_rank[t].append(ave_rank_)
                ave_mtc[t] += mtc[0][ii]
                n += 1
                m[t].append(mtc[0][ii])
        ave_mtc[t] /= n
    return NDCG, Ave_rank, ave_mtc, m

def evaluate_item(mtc, k):
    bt = int(len(mtc[0])/k)
    labels = []
    for t in range(k):
        if t == k-1:
            labels[t*bt::] = [t]*(len(mtc[0])-(k-1)*bt)
        else:
            labels[t*bt:t*(bt+1)] = [t]*bt
    
    performance = {}
    ave_mtc = {}
    m = {}
    for t in range(k):
        performance[t] = []
        ave_mtc[t] = 0
        n = 0
        m[t] = []
        for ii in range(len(labels)):
            if labels[ii] == t:
                performance[t].append(get_performance(mtc[1][ii]))
                ave_mtc[t] += mtc[0][ii]
                n += 1
                m[t].append(mtc[0][ii])
        ave_mtc[t] /= n
    return performance, ave_mtc, m


In [ ]:
### K is the number of groups
k = 5
bt_user = int(Rec.shape[0]/k)
labels = []
for t in range(k):
    if t == k-1:
        labels[t*bt_user::] = [t]*(len(usrmain_sort[0])-(k-1)*bt_user)
    else:
        labels[t*bt_user:t*(bt_user+1)] = [t]*bt_user
        

In [19]:
groups = 5
### here we do not need to calculate 4 times, call 1 time is enough
res_usrmain, ave_usrmain, m_usrmain = evaluate_user(usrmain_sort, groups)
res_act, ave_act, m_act = evaluate_user(act_sort, groups)
res_itmmain, ave_itmmain, m_itmmain = evaluate_item(itmmain_sort, groups)
res_pop, ave_pop, m_pop = evaluate_item(pop_sort, groups)

TypeError: object of type 'numpy.float64' has no len()

In [42]:
### Convert results to dataframe
def to_df(r, a, m):
    df = {"result": [], "metric": [], "group": [], "ave_metric": []}
    k = len(r)
    for t in range(k):
        l = len(r[t])
        for tt in range(l):
            df["result"].append(r[t][tt])
            df["metric"].append(m[t][tt])
            df["group"].append(t)
            df["ave_metric"].append(a[t])
    df = pd.DataFrame(df)
    return df

In [43]:
### Save the dataframes
df = to_df(res_pop, ave_pop, m_pop)
df.to_csv("./data/item pop.csv")
df = to_df(res_usrmain, ave_usrmain, m_usrmain)
df.to_csv("./data/user main.csv")
df = to_df(res_itmmain, ave_itmmain, m_itmmain)
df.to_csv("./data/item main.csv")
df = to_df(res_act, ave_act, m_act)
df.to_csv("./data/user act.csv")


In [12]:
df = to_df(res_act, ave_act, m_act)
df.to_csv("./data/user act.csv")